# BERT를 활용한 Dense Passage Retrieval 실습

### Requirements

In [1]:
# > /dev/null 2>&1 # execute command in silence
!pip install datasets==1.4.1 > /dev/null 2>&1 # execute command in silence
!pip install transformers==4.20.1 > /dev/null 2>&1

������ ��θ� ã�� �� �����ϴ�.
������ ��θ� ã�� �� �����ϴ�.


## 데이터셋 로딩


KorQuAD train 데이터셋을 학습 데이터로 활용

In [2]:
from datasets import load_dataset

dataset = load_dataset("squad_kor_v1")

Reusing dataset squad_kor_v1 (C:\Users\AI_15\.cache\huggingface\datasets\squad_kor_v1\squad_kor_v1\1.0.0\18d4f44736b8ee85671f63cb84965bfb583fa0a4ff2df3c2e10eee9693796725)


  0%|          | 0/2 [00:00<?, ?it/s]

In [3]:
corpus = list(set([example['context'] for example in dataset['train']]))
len(corpus)

9606

## 토크나이저 준비 - Huggingface 제공 tokenizer 이용

BERT를 encoder로 사용하므로, hugginface에서 제공하는 "bert-base-multilingual-cased" tokenizer를 활용

In [4]:
from transformers import AutoTokenizer
import numpy as np

# model_checkpoint = "bert-base-multilingual-cased" # out of memory issue
model_checkpoint = "kykim/bert-kor-base"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Downloading:   0%|          | 0.00/80.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/725 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/336k [00:00<?, ?B/s]

In [5]:
tokenizer

PreTrainedTokenizerFast(name_or_path='kykim/bert-kor-base', vocab_size=42000, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})

In [6]:
tokenized_input = tokenizer(dataset['train'][0]['context'], padding="max_length", truncation=True)
tokenizer.decode(tokenized_input['input_ids'])

'[CLS] 1839년 바그너는 괴테의 파우스트을 처음 읽고 그 내용에 마음이 끌려 이를 소재로 해서 하나의 교향곡을 쓰려는 뜻을 갖는다. 이 시기 바그너는 1838년에 빛 독촉으로 산전수전을 다 걲은 상황이라 좌절과 실망에 가득했으며 메피스토펠레스를 만나는 파우스트의 심경에 공감했다고 한다. 또한 파리에서 아브네크의 지휘로 파리 음악원 관현악단이 연주하는 베토벤의 교향곡 9번을 듣고 깊은 감명을 받았는데, 이것이 이듬해 1월에 파우스트의 서곡으로 쓰여진 이 작품에 조금이라도 영향을 끼쳤으리라는 것은 의심할 여지가 없다. 여기의 라단조 조성의 경우에도 그의 전기에 적혀 있는 것처럼 단순한 정신적 피로나 실의가 반영된 것이 아니라 베토벤의 합창교향곡 조성의 영향을 받은 것을 볼 수 있다. 그렇게 교향곡 작곡을 1839년부터 40년에 걸쳐 파리에서 착수했으나 1악장을 쓴 뒤에 중단했다. 또한 작품의 완성과 동시에 그는 이 서곡 ( 1악장 ) 을 파리 음악원의 연주회에서 연주할 파트보까지 준비하였으나, 실제로는 이루어지지는 않았다. 결국 초연은 4년 반이 지난 후에 드레스덴에서 연주되었고 재연도 이루어졌지만, 이후에 그대로 방치되고 말았다. 그 사이에 그는 리엔치와 방황하는 네덜란드인을 완성하고 탄호이저에도 착수하는 등 분주한 시간을 보냈는데, 그런 바쁜 생활이 이 곡을 잊게 한 것이 아닌가 하는 의견도 있다. [SEP] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PA

## Dense encoder (BERT) 학습 시키기

HuggingFace BERT를 활용하여 question encoder, passage encoder 학습

In [8]:
from tqdm import tqdm, trange
import argparse
import random
import torch
import torch.nn.functional as F
from transformers import BertModel, BertPreTrainedModel, AdamW, TrainingArguments, get_linear_schedule_with_warmup

torch.manual_seed(2021)
torch.cuda.manual_seed(2021)
np.random.seed(2021)
random.seed(2021)

1) Training Dataset 준비하기 (question, passage pairs)

---



In [9]:
# Use subset (10 example) of original training dataset 
sample_idx = np.random.choice(range(len(dataset['train'])), 20)
training_dataset = dataset['train'][sample_idx]

Negative sampling을 위한 negative sample들을 샘플링

In [10]:
# set number of neagative sample
num_neg = 3

corpus = np.array(corpus)
p_with_neg = []

for c in training_dataset['context']:
  while True:
    neg_idxs = np.random.randint(len(corpus), size=num_neg)

    if not c in corpus[neg_idxs]:
      p_neg = corpus[neg_idxs]

      p_with_neg.append(c)
      p_with_neg.extend(p_neg)
      break


In [11]:
print('[Positive context]')
print(p_with_neg[0], '\n')
print('[Negative context]')
print(p_with_neg[1], '\n', p_with_neg[2])

[Positive context]
무한도전은 회를 거듭할수록 소재가 진부해진다는 평을 듣기도 한다. 대표적인 예가 지난 2008년 1월 5일 방송된 무한도전 새해 특집이다. 바스켓을 타고 올라가는 것에 대해 두려워하는 것이 같은 방송사 프로그램 일요일 일요일 밤에의 코너인 〈불가능은 없다〉와 흡사하다는 내용이었다. 또, 일본의 프로그램을 표절하였다는 의혹도 받기도 하였다. 그 예로 2007년 12월 15일 방송된 달력 만들기 특집이 일본 요미우리 TV의 프로그램 "하마찬토"의 “2007년판 남자의 달력 인(in) 베트남”과 흡사하다는 의혹과, 2007년 11월에 방송된 대체 에너지 특집의 자전거 발전이 니혼 TV의 프로그램인 "디노아라시"의 “자전거 발전 시험”과 흡사하다는 의혹도 받고 있다. 그 밖에도 뺨을 때리는 기계와 모래공 차기, 브레이크 고장 난 자전거 타기 등은 일본의 후지 TV와 TBS의 프로그램을 표절하였다는 의혹을 받기도 하였다. 

[Negative context]
데이비드 베컴과 빅토리아 애덤스의 슬하에는 네 자식을 두고 있다: 세 아들 브루클린 조지프, (1999년 3월 4일, 런던 웨스트민스터 출생) 로미오 제임스, (2002년 9월 1일, 런던 웨스트민스터 출생) 크루스 다비드, (2005년 2월 20일, 스페인 마드리드 지방 마드리드 출생) 그리고 딸 하퍼 세븐 (2011년 7월 10일, 미국 캘리포니아 주 로스앤젤레스 출생) 이 그들이다. 엘튼 존이 브루클린과 로미오 베컴의 대부를 맡았다. 그들의 대모는 엘리자베스 헐리였다. 베컴의 세 아들 모두 아스널 유소년부에서 축구를 했다. 부친과 마찬가지로, 브루클린과 로미오 모두 모델일을 한 적이 있고, GQ에 영국의 가장 옷을 잘 입는 남자들로 명명되기도 했다. 브루클린은 아스널 U-16부에서 축구를 한 경력이 있지만, 2014-15 시즌 이후에 방출되었다. 
 하루아침에 잘나가는 검사에서 범죄자로 전락한 재욱, 그는 자신의 결백을 주장하기 위해 몇번이나 재심신청을 했지만 당연히 모두 기

In [12]:
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset)

q_seqs = tokenizer(training_dataset['question'], padding="max_length", truncation=True, return_tensors='pt')
p_seqs = tokenizer(p_with_neg, padding="max_length", truncation=True, return_tensors='pt')


In [13]:
max_len = p_seqs['input_ids'].size(-1)
p_seqs['input_ids'] = p_seqs['input_ids'].view(-1, num_neg+1, max_len)
p_seqs['attention_mask'] = p_seqs['attention_mask'].view(-1, num_neg+1, max_len)
p_seqs['token_type_ids'] = p_seqs['token_type_ids'].view(-1, num_neg+1, max_len)

print(p_seqs['input_ids'].size())  #(num_example, pos + neg, max_len)

torch.Size([20, 4, 512])


In [14]:
train_dataset = TensorDataset(p_seqs['input_ids'], p_seqs['attention_mask'], p_seqs['token_type_ids'], 
                        q_seqs['input_ids'], q_seqs['attention_mask'], q_seqs['token_type_ids'])

2) BERT encoder 학습시키기

BertEncoder 모델 정의 후, question encoder, passage encoder에 pre-trained weight 불러오기

In [15]:
class BertEncoder(BertPreTrainedModel):
  def __init__(self, config):
    super(BertEncoder, self).__init__(config)

    self.bert = BertModel(config)
    self.init_weights()
      
  def forward(self, input_ids, 
              attention_mask=None, token_type_ids=None): 
  
      outputs = self.bert(input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids)
      
      pooled_output = outputs[1]

      return pooled_output


In [16]:
# load pre-trained model on cuda (if available)
p_encoder = BertEncoder.from_pretrained(model_checkpoint)
q_encoder = BertEncoder.from_pretrained(model_checkpoint)

if torch.cuda.is_available():
  p_encoder.cuda()
  q_encoder.cuda()

Downloading:   0%|          | 0.00/454M [00:00<?, ?B/s]

Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertEncoder: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at kykim/bert-kor-base were not used when initializing BertEncoder: ['cls.pred

Train function 정의 후, 두개의 encoder fine-tuning 하기 (In-batch negative 활용) 


In [17]:
def train(args, num_neg, dataset, p_model, q_model):
  
  # Dataloader
  train_sampler = RandomSampler(dataset)
  train_dataloader = DataLoader(dataset, sampler=train_sampler, batch_size=args.per_device_train_batch_size)

  # Optimizer
  no_decay = ['bias', 'LayerNorm.weight']
  optimizer_grouped_parameters = [
        {'params': [p for n, p in p_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in p_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0},
        {'params': [p for n, p in q_model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': args.weight_decay},
        {'params': [p for n, p in q_model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]
  optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
  t_total = len(train_dataloader) // args.gradient_accumulation_steps * args.num_train_epochs
  scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=args.warmup_steps, num_training_steps=t_total)

  # Start training!
  global_step = 0
  
  p_model.zero_grad()
  q_model.zero_grad()
  torch.cuda.empty_cache()
  
  train_iterator = trange(int(args.num_train_epochs), desc="Epoch")

  for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration")

    for step, batch in enumerate(epoch_iterator):
      q_encoder.train()
      p_encoder.train()
      
      targets = torch.zeros(args.per_device_train_batch_size).long()
      if torch.cuda.is_available():
        batch = tuple(t.cuda() for t in batch)
        targets = targets.cuda()

      p_inputs = {'input_ids': batch[0].view(
                                    args.per_device_train_batch_size*(num_neg+1), -1),
                  'attention_mask': batch[1].view(
                                    args.per_device_train_batch_size*(num_neg+1), -1),
                  'token_type_ids': batch[2].view(
                                    args.per_device_train_batch_size*(num_neg+1), -1)
                  }
      
      q_inputs = {'input_ids': batch[3],
                  'attention_mask': batch[4],
                  'token_type_ids': batch[5]}
      
      p_outputs = p_model(**p_inputs)  #(batch_size*(num_neg+1), emb_dim)
      q_outputs = q_model(**q_inputs)  #(batch_size*, emb_dim)

      # Calculate similarity score & loss
      p_outputs = p_outputs.view(args.per_device_train_batch_size, -1, num_neg+1)
      q_outputs = q_outputs.view(args.per_device_train_batch_size, 1, -1)

      sim_scores = torch.bmm(q_outputs, p_outputs).squeeze()  #(batch_size, num_neg+1)
      sim_scores = sim_scores.view(args.per_device_train_batch_size, -1)
      sim_scores = F.log_softmax(sim_scores, dim=1)

      loss = F.nll_loss(sim_scores, targets)
      print(loss)

      loss.backward()
      optimizer.step()
      scheduler.step()
      q_model.zero_grad()
      p_model.zero_grad()
      global_step += 1
      
      torch.cuda.empty_cache()


    
  return p_model, q_model




In [40]:
args = TrainingArguments(
    output_dir="dense_retireval",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=2, # For demonstration
    weight_decay=0.01
)


In [41]:
# 훈련이 되어있는 모델 사용
p_encoder, q_encoder = train(args, num_neg, train_dataset, p_encoder, q_encoder)

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]

Iteration:   0%|          | 0/10 [00:00<?, ?it/s]

Epoch:   0%|          | 0/2 [00:00<?, ?it/s]


RuntimeError: CUDA out of memory. Tried to allocate 12.00 MiB (GPU 0; 8.00 GiB total capacity; 6.31 GiB already allocated; 4.83 MiB free; 6.39 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

## Dense Embedding을 활용하여 passage retrieval 실습해보기

In [20]:
valid_corpus = list(set([example['context'] for example in dataset['validation']]))[:10]
sample_idx = random.choice(range(len(dataset['validation'])))
query = dataset['validation'][sample_idx]['question']
ground_truth = dataset['validation'][sample_idx]['context']

if not ground_truth in valid_corpus:
  valid_corpus.append(ground_truth)

print(query)
print(ground_truth, '\n\n')

# valid_corpus

유아인에게 타고난 배우라고 말한 드라마 밀회의 감독은?
화보 촬영을 위해 미국에 있을 때, 김희애의 연락을 통해 JTBC 드라마 《밀회》의 캐스팅을 제안받았다. 당시 영화 《베테랑》에 이미 캐스팅된 상태였으나, 유아인은 류승완 감독과 제작사의 양해를 얻어 《밀회》에 출연한다. 천재 피아니스트 ‘이선재’ 역할을 위해 피아니스트들의 영상을 보고 곡의 스피드와 건반 위치 등을 외워 실제 타건을 하며 촬영했다. 피아노 울림판을 수건으로 막고 타건을 하면, 그 후 대역 피아니스트의 소리를 덧입히는 방식이었다. 《밀회》는 작품성을 인정받고 숱한 화제를 낳으며 당시 종편으로서는 높은 시청률을 기록했다. 유아인은 섬세한 연기력을 선보여 순수함으로 시청자들을 매료시켰다는 호평을 얻었고, 특히 피아노 연주에 있어서 클래식 종사자들에게 인정을 받았다. 연출을 맡은 안판석 감독은 유아인에 대해 “느낌으로만 연기를 하는 게 아니고 감성을 지적으로 통제해 가면서 연기한다. 그 나이에”라며 “타고난 배우”라고 말했다. 유아인은 《밀회》를 통해 예술적인 면모를 구체화할 수 있어서 만족감을 느꼈다고 밝혔으며, 종영 후 자신의 페이스북 계정에 긴 소감글을 남겼다. 특히 ‘이선재’ 캐릭터를 배우 유아인이 가진 소년성의 엑기스로 생각하며, 2015년 10월 부산국제영화제 오픈토크에서는 본인이 가장 좋아하는 캐릭터로 꼽았다. 




앞서 학습한 passage encoder, question encoder을 이용해 dense embedding 생성

In [21]:
def to_cuda(batch):
  return tuple(t.cuda() for t in batch)

In [22]:
with torch.no_grad():
  p_encoder.eval()
  q_encoder.eval()

  q_seqs_val = tokenizer([query], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
  q_emb = q_encoder(**q_seqs_val).to('cpu')  #(num_query, emb_dim)

  p_embs = []
  for p in valid_corpus:
    p = tokenizer(p, padding="max_length", truncation=True, return_tensors='pt').to('cuda')
    p_emb = p_encoder(**p).to('cpu').numpy()
    p_embs.append(p_emb)

p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)

print(p_embs.size(), q_emb.size())

torch.Size([11, 768]) torch.Size([1, 768])


C:\Users\AI_15\AppData\Local\Temp\ipykernel_15848\1112144075.py:14: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:204.)
  p_embs = torch.Tensor(p_embs).squeeze()  # (num_passage, emb_dim)


생성된 embedding에 dot product를 수행 => Document들의 similarity ranking을 구함

In [23]:
dot_prod_scores = torch.matmul(q_emb, torch.transpose(p_embs, 0, 1))
print(dot_prod_scores.size())

rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
print(dot_prod_scores)
print(rank)

torch.Size([1, 11])
tensor([[ 48.7925, 101.2781, -30.6381, 138.7635,  39.2943, 139.0693, 136.8548,
          37.1513,  11.3480,  -5.7385, 105.6256]])
tensor([ 5,  3,  6, 10,  1,  0,  4,  7,  8,  9,  2])


Top-5개의 passage를 retrieve 하고 ground truth와 비교하기

In [24]:
k = 5
print("[Search query]\n", query, "\n")
print("[Ground truth passage]")
print(ground_truth, "\n")

for i in range(k):
  print("Top-%d passage with score %.4f" % (i+1, dot_prod_scores.squeeze()[rank[i]]))
  print(valid_corpus[rank[i]])

[Search query]
 유아인에게 타고난 배우라고 말한 드라마 밀회의 감독은? 

[Ground truth passage]
화보 촬영을 위해 미국에 있을 때, 김희애의 연락을 통해 JTBC 드라마 《밀회》의 캐스팅을 제안받았다. 당시 영화 《베테랑》에 이미 캐스팅된 상태였으나, 유아인은 류승완 감독과 제작사의 양해를 얻어 《밀회》에 출연한다. 천재 피아니스트 ‘이선재’ 역할을 위해 피아니스트들의 영상을 보고 곡의 스피드와 건반 위치 등을 외워 실제 타건을 하며 촬영했다. 피아노 울림판을 수건으로 막고 타건을 하면, 그 후 대역 피아니스트의 소리를 덧입히는 방식이었다. 《밀회》는 작품성을 인정받고 숱한 화제를 낳으며 당시 종편으로서는 높은 시청률을 기록했다. 유아인은 섬세한 연기력을 선보여 순수함으로 시청자들을 매료시켰다는 호평을 얻었고, 특히 피아노 연주에 있어서 클래식 종사자들에게 인정을 받았다. 연출을 맡은 안판석 감독은 유아인에 대해 “느낌으로만 연기를 하는 게 아니고 감성을 지적으로 통제해 가면서 연기한다. 그 나이에”라며 “타고난 배우”라고 말했다. 유아인은 《밀회》를 통해 예술적인 면모를 구체화할 수 있어서 만족감을 느꼈다고 밝혔으며, 종영 후 자신의 페이스북 계정에 긴 소감글을 남겼다. 특히 ‘이선재’ 캐릭터를 배우 유아인이 가진 소년성의 엑기스로 생각하며, 2015년 10월 부산국제영화제 오픈토크에서는 본인이 가장 좋아하는 캐릭터로 꼽았다. 

Top-1 passage with score 139.0693
김태희는 이후 지하철에서 광고대행사 디자이너를 우연히 만나 2000년 "화이트" 광고를 찍으면서 연예계에 데뷔했다. 광고 촬영 이후 여러 곳에서 모델 제의가 들어왔고 그녀가 서울대학교 출신이라는 점 때문에 더 화제가 되었다. 2001년 영화 《선물》에서 중등 정연 역을 맡으며 스크린에 첫 데뷔했다. 당시 영화 감독 오기환은 잡지 표지모델로 섰던 김태희를 보고 테스트도 안 해도되니 그냥 데려 오라고 말했다. 이후 대학생활의 숙원이었던 어학연수

## Wikipedia documents에 대해 Dense Passage Retrieval 실습하기

In [25]:
!wg --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd" -O wikipedia_documenet --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quietts.json && rm -rf /tmp/cookies.txt

Cannot open cookies file '/tmp/cookies.txt': No such file or directory
--2022-07-28 11:21:48--  https://docs.google.com/uc?export=download&confirm=$(wget%20--quiet%20--save-cookies%20/tmp/cookies.txt%20--keep-session-cookies%20--no-check-certificate%20'https://docs.google.com/uc?export=download&id=1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd'%20-O-%20%7C%20sed%20-rn%20's/.*confirm=([0-9A-Za-z_]+).*/%5C1%5Cn/p')&id=1wVIgtc0YoQEwXB3JAsUud_86fRzrFCBd
Resolving docs.google.com (docs.google.com)... 142.250.207.110
Connecting to docs.google.com (docs.google.com)|142.250.207.110|:443... connected.
HTTP request sent, awaiting response... 404 Not Found
2022-07-28 11:21:49 ERROR 404: Not Found.



In [28]:
# First load wikipedia dump
import json

dump_path = 'wikipedia_documents.json' 
with open(dump_path, 'r', encoding='UTF-8') as f:
    wiki = json.load(f)

In [29]:
len(wiki), type(wiki)

(60613, dict)

In [30]:
wiki['0']

{'text': '이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.',
 'corpus_source': '위키피디아',
 'url': 'TODO',
 'domain': None,
 'title': '나라 목록',
 'author': None,
 'html': None,
 'document_id': 0}

In [31]:
corpus = [document['text'] for document_id, document in wiki.items()]

In [32]:
from tqdm.auto import tqdm
# wiki_embs = embedded dense vectors of documents
with torch.no_grad():
  wiki_embs = []
  for text in tqdm(corpus):
    p = tokenizer(text, padding="max_length", truncation=True, return_tensors='pt').to('cuda')
    wiki_emb = p_encoder(**p).to('cpu').numpy()
    wiki_embs.append(wiki_emb)

wiki_embs = torch.Tensor(wiki_embs).squeeze()  # (num_passage, emb_dim)

  0%|          | 0/60613 [00:00<?, ?it/s]

In [33]:
# Answer: (60613, 768) (num_doc, emb_dim)
wiki_embs.shape

torch.Size([60613, 768])

In [34]:
# TODO: embed query to dense vector
query = '대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?'
with torch.no_grad():
  q_seqs_val = tokenizer([query], padding="max_length", truncation=True, return_tensors='pt').to('cuda')
  q_emb = q_encoder(**q_seqs_val).to('cpu')  #(num_query, emb_dim)

In [35]:
result = torch.matmul(q_emb, torch.transpose(wiki_embs, 0, 1))
result.shape

torch.Size([1, 60613])

In [36]:
k = 5
rank = torch.argsort(result, dim=1, descending=True).squeeze()

In [37]:
k = 5
print("[Search query]\n", query, "\n")

for i in range(k):
  print("Top-%d passage with score %.4f" % (i+1, result.squeeze()[rank[i]]))
  print(corpus[rank[i]])

[Search query]
 대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은? 

Top-1 passage with score 212.0865
야음초등학교  
약사초등학교  
약수초등학교  
양사초등학교  
양지초등학교  
언양초등학교  
여천초등학교  
연암초등학교  
염포초등학교  
영화초등학교  
옥동초등학교  
옥산초등학교  
옥서초등학교  
옥성초등학교  
옥현초등학교  
온남초등학교  
온산초등학교  
온양초등학교  
용연초등학교  
우정초등학교  
울산남부초등학교  
울산양정초등학교  
울산중앙초등학교  
울산초등학교  
울주명지초등학교  
웅촌초등학교  
웅촌초등학교 검단분교  
월계초등학교  
월봉초등학교  
월평초등학교  
이화초등학교  
일산초등학교  
외솔초등학교
Top-2 passage with score 208.5267
자인초등학교 
자천초등학교 보현분교 
자천초등학교 
장곡초등학교 
장기초등학교 
장기초등학교 모포분교 
장량초등학교 
장산초등학교 
장성초등학교 
장수초등학교 
장천초등학교 
재산초등학교 
저동초등학교 
점곡초등학교 
점촌북초등학교 
점촌중앙초등학교 
점촌초등학교 
정수초등학교 
정평초등학교 
조마초등학교 
죽도초등학교 
죽변초등학교 
죽장초등학교 상옥분교 
죽장초등학교 
죽천초등학교 
중동초등학교 
중모초등학교 
증산초등학교 
지곡초등학교 
지동초등학교 
지례초등학교 
지방초등학교 
지보초등학교 
지사초등학교 
지산초등학교 
지천초등학교 
지품초등학교 
직지초등학교 
진량초등학교 
진보초등학교 
진성초등학교 
진평초등학교
Top-3 passage with score 207.2272
자은초등학교 
작천초등학교 
장동초등학교 
장산초등학교 
장성성산초등학교 
장성중앙초등학교 
장천초등학교 
장평초등학교 
장흥남초등학교 
장흥서초등학교 
장흥초등학교 
점암초등학교 신안분교 
점암초등학교 화계분교 
점암초등학교 
조도초등학교 거차분교 
조도초등학교 관사분교 
조도초등학교 대마분교 
조도초등학교 
조성남초등학교 
조성

In [42]:
!nvidia-smi

Thu Jul 28 13:36:57 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 451.67       Driver Version: 451.67       CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 207... WDDM  | 00000000:01:00.0  On |                  N/A |
| N/A   49C    P0    28W /  N/A |   7813MiB /  8192MiB |      2%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                                  |
|  GPU  